## 1. Introduction
<p><a href="https://commons.wikimedia.org/wiki/File:Tux.svg">
<img style="float: right;margin:5px 20px 5px 1px" width="150px" src="https://assets.datacamp.com/production/project_111/img/tux.png" alt="Tux: la mascota de Linux">
</a></p>
<p>Los repositorios de control de versiones como CVS, Subversion o Git pueden ser una verdadera mina de oro para los desarrolladores de software. Contienen cada cambio realizado al código fuente, incluyendo la fecha (el "cuándo"), el desarrollador responsable (el "quién"), así como un pequeño mensaje que describe la intención (el "qué") de un cambio.</p>
<p>En este cuaderno, analizaremos la evolución de un proyecto de código abierto muy famoso: el kernel de Linux. El kernel de Linux es el corazón de algunas distribuciones de Linux como Debian, Ubuntu o CentOS. Nuestro conjunto de datos contiene el historial del desarrollo del kernel durante casi 13 años (principios de 2005 - finales de 2017). Obtendremos información sobre el trabajo de los esfuerzos de desarrollo mediante:</p>
<ul>
<li>Identificar a los 10 principales contribuyentes </li>
<li>Visualizar los commits a lo largo de los años.</li>
</ul>

In [5]:
# Printing the content of git_log_excerpt.csv
# ... YOUR CODE FOR TASK 1 ...

## 2. Reading in the dataset
<p>El conjunto de datos fue creado utilizando el comando <code>git log --encoding=latin-1 --pretty="%at#%aN"</code> a finales de 2017. La salida de texto codificada en <code>latin-1</code> se guardó en un archivo CSV sin encabezado. En este archivo, cada fila es una entrada de commit con la siguiente información:</p>
<ul>
<li><code>timestamp</code>: la hora del commit como una marca de tiempo UNIX en segundos desde 1970-01-01 00:00:00 (Marcador de posición de Git log "<code>%at</code>")</li>
<li><code>author</code>: el nombre del autor que realizó el commit (Marcador de posición de Git log "<code>%aN</code>")</li>
</ul>
<p>Las columnas están separadas por el signo de número <code>#</code>. El conjunto de datos completo se encuentra en el directorio <code>datasets/</code>. Es un archivo csv comprimido en <code>gz</code> llamado <code>git_log.gz</code>.</p>

In [ ]:
# Loading in the pandas module as 'pd'
# ... YOUR CODE FOR TASK 2 ...

# Reading in the log file
git_log = ...

# Printing out the first 5 rows
# ... YOUR CODE FOR TASK 2 ...

## 3. Obteniendo una visión general
<p>El conjunto de datos contiene información sobre cada contribución de código (un "commit") al kernel de Linux en los últimos 13 años. Primero echaremos un vistazo al número de autores y sus commits en el repositorio.</p>

In [ ]:
# calculating number of commits
number_of_commits = ...

# calculating number of authors
number_of_authors = ...

# printing out the results
print("%s authors committed %s code changes." % (number_of_authors, number_of_commits))

## 4. Encontrando los 10 principales contribuyentes
<p>Existen algunas personas muy importantes que han realizado cambios frecuentes en el kernel de Linux. Para ver si hay algún cuello de botella, vamos a analizar a los 10 principales autores con más commits.</p>

In [ ]:
# Identifying the top 10 authors
top_10_authors = ...

# Listing contents of 'top_10_authors'
top_10_authors

## 5. Manipulando los datos
<p>Para nuestro análisis, queremos visualizar las contribuciones a lo largo del tiempo. Para esto, utilizaremos la información en la columna <code>timestamp</code> para crear una columna basada en series de tiempo.</p>

In [ ]:
# converting the timestamp column
# ... YOUR CODE FOR TASK 5 ...

# summarizing the converted timestamp column
# ... YOUR CODE FOR TASK 5 ...

## 6. Tratando los timestamps incorrectos
<p>Como podemos ver en los resultados anteriores, algunos contribuyentes tenían incorrectamente configurada la hora de su sistema operativo cuando realizaron los commits en el repositorio. Vamos a limpiar la columna <code>timestamp</code> eliminando las filas con timestamps incorrectos.</p>

In [ ]:
# determining the first real commit timestamp
first_commit_timestamp = ...

# determining the last sensible commit timestamp
last_commit_timestamp = ...

# filtering out wrong timestamps
corrected_log = ...

# summarizing the corrected timestamp column
# ... YOUR CODE FOR TASK 6 ...

## 7. Agrupando los commits por año
<p>Para determinar cómo ha aumentado la actividad de desarrollo a lo largo del tiempo, vamos a agrupar los commits por año y contarlos.</p>

In [ ]:
# Counting the no. commits per year
commits_per_year = ...

# Listing the first rows
# ... YOUR CODE FOR TASK 7 ...

## 8. Visualizando la historia de Linux
<p>Finalmente, crearemos un gráfico a partir de estos recuentos para poder visualizar mejor cómo ha aumentado el esfuerzo de desarrollo en Linux en los últimos años.</p>

In [ ]:
# Setting up plotting in Jupyter notebooks
%matplotlib inline

# plot the data
# ... YOUR CODE FOR TASK 8 ...

## 9. Conclusión
<p>Gracias a la sólida base y al cuidado de Linux Torvalds, muchos otros desarrolladores ahora también pueden contribuir al kernel de Linux. ¡No se observa una disminución de la actividad de desarrollo a la vista!</p>

In [ ]:
# calculating or setting the year with the most commits to Linux
year_with_most_commits = ... 